# Using AI and NLP to Improve Your Cooking

Michael Humkey, Conor Marsten

In [89]:
import urllib

In [90]:
url = "http://food2fork.com/api/search?"
args = {'key' : 'd8b2df92a9cb994f2009b2be8410c1a3', 'q' : 'chicken', 'sort' : 'r'}
data = urllib.parse.urlencode(args)
req = urllib.request.Request(url+data, headers={'User-Agent': 'Mozilla/5.0'})
response = urllib.request.urlopen(req).read()

In [91]:
import json

In [92]:
response = json.loads(response)
recipes = response['recipes']

In [93]:
from bs4 import BeautifulSoup

In [96]:
def getReviews(recipes, i=0):

    recipe = recipes[i]
    url = "http://food2fork.com/api/get?"
    data = urllib.parse.urlencode({'key' : 'd8b2df92a9cb994f2009b2be8410c1a3', 'rId' : recipe['recipe_id']})
    req = urllib.request.Request(url+data, headers={'User-Agent': 'Mozilla/5.0'})
    recipe = json.loads(urllib.request.urlopen(req).read())['recipe']
    bs =  BeautifulSoup(requests.get(recipe['source_url']).content, 'html.parser')
    reviews = []

    if (recipe['publisher'] == '101 Cookbooks'): #1
        reviews = bs.find_all(attrs={'class': 'card-body'})

    elif (recipe['publisher'] == 'BBC Good Food'):
        reviews = bs.find_all(attrs={'class': 'field-item even'})

    elif (recipe['publisher'] == 'Closet Cooking' 
           or recipe['publisher'] == 'Eats Well With Others' 
           or recipe['publisher'] == "Lisa's Kitchen"
           or recipe['publisher'] == 'A Spicy Perspective'
           or recipe['publisher'] == 'Naturally Ella'
           or recipe['publisher'] == 'Pastry Affair'):
        reviews = bs.find_all(attrs={'class': 'comment-body'})

    elif (recipe['publisher'] == 'Food Republic'):
        reviews = bs.find_all(attrs={'class': '_5mdd'}) #uses facebook comments plugin

    elif (recipe['publisher'] == 'PBS Food'):
        reviews = bs.find_all(attrs={'class': 'post-message'})

    elif (recipe['publisher'] == 'Simply Recipes'
          or recipe['publisher'] == 'Homesick Texan'
          or recipe['publisher'] == 'Tasty Kitchen'):
        reviews = bs.find_all(attrs={'class': 'comment-text'})

    elif (recipe['publisher'] == 'Two Peas and Their Pod'
          or recipe['publisher'] == 'Cookie and Kate'
          or recipe['publisher'] == "Elana's Pantry"
          or recipe['publisher'] == 'My Baking Addiction'
          or recipe['publisher'] == 'Smitten Kitchen'
          or recipe['publisher'] == 'Vintage Mixer'
          or recipe['publisher'] == 'Cookin Canuck'
          or recipe['publisher'] == 'Healthy Delicious'
          or recipe['publisher'] == 'Steamy Kitchen'
          or recipe['publisher'] == "What's Gaby Cooking"
          or recipe['publisher'] == 'Bunky Cooks'
          or recipe['publisher'] == 'Serious Eats'):
        reviews = bs.find_all(attrs={'class': 'comment-content'})

    elif (recipe['publisher'] == 'All Recipes'):
        reviews = bs.find_all(itemprop="reviewBody")

    elif (recipe['publisher'] == 'Big Girls Small Kitchen'
          or recipe['publisher'] =='Jamie Oliver'
          or recipe['publisher'] == 'The Pioneer Woman'):
        reviews = bs.find_all(attrs={'data-role': 'message'})

    elif (recipe['publisher'] == 'Framed Cooks'
          or recipe['publisher'] == 'Picky Palate'):
        reviews = bs.find_all(attrs={'class': 'comment even thread-even depth-1'})

    elif (recipe['publisher'] == 'Bon Appetit'):
        reviews = bs.find_all(attrs={'class': 'review-body'})

    elif (recipe['publisher'] == 'Epicurious'):
        reviews = bs.find_all(attrs={'class': 'review-text'})

    elif (recipe['publisher'] == 'Cookstr'):
        reviews = bs.find_all(attrs={'class': 'commentText'})

    elif (recipe['publisher'] == 'Panini Happy'):
        #careful, sloppy html
        reviews = bs.find_all(attrs={'class': 'format_text'})

    elif (recipe['publisher'] == 'Real Simple'):
        reviews = bs.find_all(attrs={'class': 'comment_txt'})

    elif (recipe['publisher'] == 'BBC Food'):
        #DOES NOT HAVE REVIEWS
        print('TOSS THIS ONE BACK IT AINT GOT NO REVIEWS\n')

    elif (recipe['publisher'] == 'Chow'):
        reviews = bs.find_all(itemprop="comment")

    elif (recipe['publisher'] == 'Delishhh'):
        reviews = bs.find_all(attrs={'class': 'commentmeta'})

    elif (recipe['publisher'] == 'Food Network'):
        reviews = bs.find_all(attrs={'class': 'gig-comment-body'})    
    else:
        print(recipe['publisher'])
        getReviews(recipes, i+1)
        
    return reviews

In [ ]:
reviews = getReviews(recipes)
revs = [rv.get_text().replace('\n', ' ') for rv in reviews]

In [98]:
revs

[" I HAVE to make this for my husband sometime soon.  We're trying to use up the last jalapenos from our garden this year - this would be perfect!!! Yum ",
 " see now that's the way to do it.little bit of everything to dress up the chicken ",
 ' This looks really delicious and a nice twist on a great flavor combination.  ',
 ' That sounds like dinner tomorrow night! ',
 " i grew up eating something just like this! thank you for the memory - i think it's time to revisit it! ",
 ' Intriguing, and looks delicious! ',
 ' Wonderful! I love that tasty filling.Cheers,Rosa ',
 ' What a great idea!  From the looks of your similar recipe referrals, it appears that you like jalapeno poppers as much as I do. :)  Definitely on my "must make" list! ',
 ' Awesome. I am going to have to try this soon! ',
 ' Aaaaand I just died.  ',
 ' This chicken looks fantastic Kevin - everything is better with bacon! ',
 " I'm not the biggest chicken fan, but I would totally clean my plate if I had this for dinner!